<a href="https://colab.research.google.com/github/bhagyashreedhebe/COMP8240_MajorProject/blob/main/NewData_Pegasus_wikihow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
!pip3 install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html


In [59]:
!pip install sentencepiece

In [60]:
!pip install transformers

In [61]:
!pip install rouge-score

In [62]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
from rouge_score import rouge_scorer
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re

In [63]:
model_name = 'google/pegasus-wikihow'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

In [64]:
def summarize_wikihow_text(source, source_link):
        input_wikihow = []
        predicted = []
        req = Request(source_link, headers={'User-Agent': 'XYZ/3.0'})
        source_url = urlopen(req, timeout=10).read()
        # print(source_url)
        soup = BeautifulSoup(source_url,'lxml')
        text = ''
        for paragraph in soup.find_all('p'):
            text += paragraph.text
        text = re.sub(r'\[.*?\]+', '', text)
        text = text.replace('\n', ' ')
        text = ' '.join(text.split())
        input_wikihow.append(text)
        input = ' '.join(map(str, input_wikihow))
        batch = tokenizer(input, truncation=True, padding='longest', return_tensors="pt").to(device)
        translated = model.generate(**batch)
        tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
        predicted = ' '.join(map(str, tgt_text)) 
        model_trained = 'pegasus-wikihow'
        return model_trained, source, predicted

In [65]:
wikihow_urls = ['https://www.wikihow.com/Set-Up-a-Tent', 'https://www.wikihow.com/Publish-a-Novel',
             'https://www.wikihow.com/Book-a-Flight', 'https://www.wikihow.com/Communicate-Effectively',
             'https://www.wikihow.com/Treat-Sudden-Cardiac-Arrest', 'https://www.wikihow.com/Doodle',
             'https://www.wikihow.com/Use-the-Rule-of-72', 'https://www.wikihow.com/Train-for-a-Triathlon',
             'https://www.wikihow.com/Use-Crop-Rotation-in-Gardening', 'https://www.wikihow.com/Work-Out-at-Home-Using-Hand-Weights']

In [66]:
import pandas as pd

dataframe = pd.DataFrame({},columns = ['model_trained_on','text_source','predicted_summary','original_summary',
                                       'recall', 'precision', 'fmeasure','human_evaluation'])

for url in wikihow_urls:
  url1 = url
  model_trained, source, wikihow_prediction  = summarize_wikihow_text('wikihow',url1)
  dataframe = dataframe.append({'model_trained_on': model_trained, 'text_source':source, 
                                'predicted_summary': wikihow_prediction
                                },ignore_index=True)

In [67]:
print('Model used for training: ', model_trained)
print('Source for the text: ', source)
print('Prediction: ',wikihow_prediction)

Model used for training:  pegasus-wikihow
Source for the text:  wikihow
Prediction:  Go to the wikiHow article How to Workout with Hand Weights by Personal Trainer co-author Monica Morris for more information on how to workout with hand weights.


In [68]:
news_urls = ['https://www.abc.net.au/news/2021-11-01/french-macron-accuses-morrison-of-lying-submarine-contract/100584196',
'https://www.bbc.com/news/world-asia-59029517',
'https://thenewdaily.com.au/news/2021/10/31/australia-indonesia-energy-climate/',
'https://www.9news.com.au/world/south-pacific-island-tonga-records-first-case-of-covid-19-prompting-people-to-get-vaccinated-and-warnings-of-a-national-lockdown/a06d7580-a160-4ad1-a2ae-d287edfba62d',
'https://www.abc.net.au/news/2021-11-01/australia-climate-change-reputation-ahead-of-cop26-warming-talks/100579386',
'https://www.9news.com.au/national/coronavirus-international-travel-update-november-1-australian-borders-open-many-still-cant-get-home-exclusive/6a1ac54e-cfaa-40e2-b1b7-9bca7c14d60f',
'https://www.theguardian.com/australia-news/2021/oct/31/new-zealand-australia-quarantine-free-travel-coronavirus',
'https://7news.com.au/news/crime/knife-acid-attack-on-japan-train-reports-c-4381302',
'https://www.abc.net.au/news/2021-10-31/qld-coronavirus-covid-zero-cases-vaccination-push/100578580',
'https://thenewdaily.com.au/news/politics/2021/11/01/scott-morrison-glasgow-cop26/']

In [69]:
for url in news_urls:
  #print(url)
  url1 = url
  model_trained, source, wikihow_prediction  = summarize_wikihow_text('news',url1)
  dataframe = dataframe.append({'model_trained_on': model_trained, 'text_source':source,
                                'predicted_summary': wikihow_prediction
                                },ignore_index=True)

In [70]:
# adding summary for the text written by the team member

dataframe['original_summary'] = ['This article summarizes the steps for finding a spot suitable for setting up a tent. Also, the steps for packing and maintaining tent. There are three sections for each step. A detailed description of the steps can be found under each section.',
                      'A guide to provide guidance for publishing a novel. Write up a novel and look out for potential agents. Send a query letter to the agent and if they are interested, they shall request for a manuscript. Keep writing while waiting for the novel to be published.',
                      'The article has steps for booking a flight online as well as booking it with an airline or travel agent. Come up with a tentative travel plan. Compare the prices offered and then choose and book a ticket.',
                      'Steps for effective communication are listed down in the article. For effective communication first step is to create a right environment. Then organize the ideas that you want to communicate. Also, how to communicate while giving a speech, communicating with body language and how to communicate during conflicts is conveyed.',
                      'A medically reviewed article for treating a sudden cardiac arrest. How to identify if someone is having a sudden cardiac arrest, providing basic life support, using automated external defibrillator are described here.',
                      'A guide for doodling. Getting right tools and learning basic patterns is the first thing that one must do. It would be good to have a variety of objects in the drawing and have a free flow of thoughts while drawing.',
                      'A rule used in finance is described in detail in the article. Estimating double time, growth rate, and exponential decay is explained. The derivation of the same has been performed.',
                      'The correct way to train for a triathlon is directed. Process for getting equipped for a race such as deciding the type of race, choosing the gear and signing up for the race is explained. How to train for becoming a triathlete is given details on.',
                      'Steps for making use of crop rotation in gardening are explained in this article. Setting up the garden and preparing it for crop rotation are described in detail.',
                      'Steps for working out with some hand weights safely at home are provided. Choosing the hand weight and physical actions are explained.',
                      'Australian PM Scott Morrison was accused of lying about a $90 billion submarine contract cancelled six weeks ago by French president Emmanuel Macron.',
                      'A news article depicting the changes in the lives of residents of Afghanistan since the Taliban took over the country.',
                      'Australia will be liaising with Indonesia for tackling climate change issues and help in advancements of low-emission technologies.',
                      'Tonga a south pacific island nation has reported its first COVID-19 positive case. The nation is gearing up to fight the spread of the virus.',
                      'Article describing how the reputation of Australia has been tarnished when it comes to climate change and measure being taken to improve it.',
                      'Article on how in spite of Australia opening its borders, many Australians aren’t able to get back to their home due to various factors.',
                      'A news article detailing plan for a quarantine free travel from New Zealand to Australia for vaccinated Australian citizens and permanent residents.',
                      'Halloween horror in Tokyo as a commuter dressed up as The Joker attacks train passengers waving a long knife and leaving 10 injured.',
                      'Queensland government is urging its resident to get fully vaccinated. It has proved a successful campaign with zero COVID-19 cases post its launch.',
                      'News article on how the Morrison government is responsible for lagging behind for efforts put in for COP26 at Glasgow.']

In [71]:
# adding human evaluation score for the text 
# human evaluation done by the team member

dataframe['human_evaluation'] = [2,4,5,1,1,2,5,4,4,5,
                                 1,2,1,4,5,1,5,4,3,3]

In [72]:
dataframe

,model_trained_on,text_source,predicted_summary,original_summary,recall,precision,fmeasure,human_evaluation
0,pegasus-wikihow,wikihow,Learn how to set up a tent with this wikiHow a...,This article summarizes the steps for finding ...,NaN,NaN,NaN,2
1,pegasus-wikihow,wikihow,Learn how to write a manuscript and find a lit...,A guide to provide guidance for publishing a n...,NaN,NaN,NaN,4
2,pegasus-wikihow,wikihow,Learn how to get from point A to point B with ...,The article has steps for booking a flight onl...,NaN,NaN,NaN,5
3,pegasus-wikihow,wikihow,Follow the steps in the How to Communicate Wit...,Steps for effective communication are listed d...,NaN,NaN,NaN,1
4,pegasus-wikihow,wikihow,Understand how this article was medically revi...,A medically reviewed article for treating a su...,NaN,NaN,NaN,1
5,pegasus-wikihow,wikihow,Learn about the authors and the wikiHow commun...,A guide for doodling. Getting right tools and ...,NaN,NaN,NaN,2
6,pegasus-wikihow,wikihow,Use the Rule of 72 to determine the number of ...,A rule used in finance is described in detail ...,NaN,NaN,NaN,5
7,pegasus-wikihow,wikihow,Read How to Be a Triathlete by the co-author(s...,The correct way to train for a triathlon is di...,NaN,NaN,NaN,4
8,pegasus-wikihow,wikihow,Follow the steps in this article to learn how ...,Steps for making use of crop rotation in garde...,NaN,NaN,NaN,4
9,pegasus-wikihow,wikihow,Go to the wikiHow article How to Workout with ...,Steps for working out with some hand weights s...,NaN,NaN,NaN,5


In [73]:
recall = []
precision = []
fmeasure = []
for i in range(0,len(dataframe['original_summary'])):
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
  scores = scorer.score(str(dataframe['original_summary'][i]),str(dataframe['predicted_summary'][i]))
  recall.append(scores['rouge1'].recall)
  precision.append(scores['rouge1'].precision)
  fmeasure.append(scores['rouge1'].fmeasure)
dataframe['recall'] =recall
dataframe['precision'] =precision
dataframe['fmeasure'] =fmeasure


In [74]:
dataframe

,model_trained_on,text_source,predicted_summary,original_summary,recall,precision,fmeasure,human_evaluation
0,pegasus-wikihow,wikihow,Learn how to set up a tent with this wikiHow a...,This article summarizes the steps for finding ...,0.238095,0.303030,0.266667,2
1,pegasus-wikihow,wikihow,Learn how to write a manuscript and find a lit...,A guide to provide guidance for publishing a n...,0.276596,0.433333,0.337662,4
2,pegasus-wikihow,wikihow,Learn how to get from point A to point B with ...,The article has steps for booking a flight onl...,0.236842,0.290323,0.260870,5
3,pegasus-wikihow,wikihow,Follow the steps in the How to Communicate Wit...,Steps for effective communication are listed d...,0.200000,0.370370,0.259740,1
4,pegasus-wikihow,wikihow,Understand how this article was medically revi...,A medically reviewed article for treating a su...,0.125000,0.129032,0.126984,1
5,pegasus-wikihow,wikihow,Learn about the authors and the wikiHow commun...,A guide for doodling. Getting right tools and ...,0.219512,0.321429,0.260870,2
6,pegasus-wikihow,wikihow,Use the Rule of 72 to determine the number of ...,A rule used in finance is described in detail ...,0.333333,0.188679,0.240964,5
7,pegasus-wikihow,wikihow,Read How to Be a Triathlete by the co-author(s...,The correct way to train for a triathlon is di...,0.173913,0.320000,0.225352,4
8,pegasus-wikihow,wikihow,Follow the steps in this article to learn how ...,Steps for making use of crop rotation in garde...,0.464286,0.361111,0.406250,4
9,pegasus-wikihow,wikihow,Go to the wikiHow article How to Workout with ...,Steps for working out with some hand weights s...,0.318182,0.250000,0.280000,5


In [76]:
dataframe.to_csv('bhag.csv',index=False)